In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 11pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 11pt; line-height: 140%;}</style>"))

In [20]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime as dt
import pycaret

ModuleNotFoundError: No module named 'pycaret'

# 데이터 로드 및 피쳐 생성

In [4]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train[['현본사소속재택근무자수', '중식계', '석식계']] = train[['현본사소속재택근무자수', '중식계', '석식계']].astype('int')
test['현본사소속재택근무자수'] = test['현본사소속재택근무자수'].astype('int')

train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.week
train['요일'] = train['일자'].dt.weekday
train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.week
test['요일'] = test['일자'].dt.weekday
test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

print('done')

done


<ipython-input-4-07497b727a46>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train['주'] = train['일자'].dt.week
<ipython-input-4-07497b727a46>:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test['주'] = test['일자'].dt.week


# 원핫인코딩으로 메뉴 분류

In [5]:
def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_



In [6]:
train['중식메뉴_split'] = train['중식메뉴'].apply(lambda x: get_food_embedding(x))
train['석식메뉴_split'] = train['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [7]:
test['중식메뉴_split'] = test['중식메뉴'].apply(lambda x: get_food_embedding(x))
test['석식메뉴_split'] = test['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [8]:
train['중식메뉴_split'][0]

['잡곡밥', '쇠불고기', '계란찜', '쌀밥', '포기김치', '청포묵무침', '요구르트', '오징어찌개']

In [9]:
lunch_lst = []
lunch_count = []
pref = 0
for i in tqdm(range(1205)):
    try:
        if train['중식계'][i] >= 880:
            pref = 1
        else:
            pref = 0
            
        lunch_lst.append(train['중식메뉴_split'][i])
        lunch_count.append(pref)
        
    except:
        pass
    
print('done')

  0%|          | 0/1205 [00:00<?, ?it/s]

done


In [10]:
lunch_lst

[['잡곡밥', '쇠불고기', '계란찜', '쌀밥', '포기김치', '청포묵무침', '요구르트', '오징어찌개'],
 ['가자미튀김', '잡곡밥', '김치찌개', '쌀밥', '마늘쫑무침', '요구르트', '모둠소세지구이', '배추겉절이'],
 ['치킨핑거', '포기김치', '팽이장국', '요구르트', '카레덮밥', '견과류조림', '쫄면야채무침'],
 ['잡곡밥', '쇠고기무국', '부추전', '시금치나물', '쌀밥', '포기김치', '주꾸미볶음', '요구르트'],
 ['잡곡밥', '우엉잡채', '쌀밥', '떡국', '포기김치', '청경채무침', '요구르트', '돈육씨앗강정'],
 ['잡곡밥', '도토리묵무침', '양파절임', '쌀밥', '포기김치', '훈제오리구이', '요구르트', '쌈무', '시래기국'],
 ['옥수수전', '유채나물', '잡곡밥', '꽃게탕', '쌀밥', '포기김치', '돈육굴소스볶음', '요구르트'],
 ['잡곡밥', '연두부*양념장', '시금치국', '쌀밥', '포기김치', '닭감자조림', '요구르트', '콩나물무침'],
 ['취나물', '탕수어', '잡곡밥', '쇠고기무국', '오징어숙회무침', '쌀밥', '포기김치', '요구르트'],
 ['냉이된장국', '잡곡밥', '쌀밥', '포기김치', '요구르트', '쇠고기장조림', '치커리무침', '통도라지구이'],
 ['고구마치즈구이', '포기김치', '해초무침', '유부장국', '요구르트', '낙지비빔밥'],
 ['대구찌개', '잡곡밥', '새송이버섯볶음', '쌀밥', '포기김치', '돌나물무침', '마늘간장치킨', '치킨무', '요구르트'],
 ['갈치구이',
  '쇠고기느타리국',
  '파래김*양념장',
  '쌀밥',
  '포기김치',
  '부럼',
  '호박꼬지&고사리&무나물',
  '요구르트',
  '오곡밥'],
 ['버섯잡채', '잡곡밥', '꽃상추무침', '쌀밥', '포기김치', '돈육간장볶음', '콩나물국', '요구르트'],
 ['소세지피망볶음', '비빔밥', '귤', '포기

In [11]:
lunch_count

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,


In [15]:
dinner_lst = []
dinner_count = []
pref_d = 0
for i in tqdm(range(1205)):
    try:
        if train['석식계'][i] > 476:
            pref_d = 1
        else:
            pref_d = 0
            
        dinner_lst.append(train['석식메뉴_split'][i])
        dinner_count.append(pref_d)
        
    except:
        pass
    
print('done')


done


In [16]:
lunch_lst_test = []
lunch_count_test = []
for i in tqdm(range(50)):
    try:
        lunch_lst_test.append(test['중식메뉴_split'][i])
        lunch_count_test.append(0)
    except:
        pass
    
print('done')


done


In [17]:
dinner_lst_test = []
dinner_count_test = []
for i in tqdm(range(50)):
    try:
        dinner_lst_test.append(test['석식메뉴_split'][i])
        dinner_count_test.append(0)
    except:
        pass

print('done')


done


In [22]:
lunch_df = pd.DataFrame({'중식메뉴':lunch_lst})
dinner_df = pd.DataFrame({'석식메뉴':dinner_lst})
lunch_df_t = pd.DataFrame({'중식메뉴':lunch_lst_test})
dinner_df_t = pd.DataFrame({'석식메뉴':dinner_lst_test})

# 메뉴 분류

In [53]:
menu = pd.read_csv('../data/pinkcoding_data_menu.csv', encoding='cp949')

In [54]:
menu.shape

(1198, 36)

In [55]:
menu.head(2)

,메뉴,곡류,두류,난류,묵류,어패류,육류,채소류,해조류,떡류,...,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류,음료.및.주류
0,가지구이양념장,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,가지볶음,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
food = list(menu['메뉴'])


In [57]:
len(food)

1198

In [58]:
food = list(menu['메뉴'])
result_lunch = []

for k in range(len(lunch_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df.iloc[k][0])):
        if lunch_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
            
    trigger = trigger / trigger.sum()
    result_lunch.append(trigger)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [59]:
len(result_lunch)

1205

In [60]:
np.isnan(result_lunch).sum()

68

In [47]:
result_dinner = []

for k in range(len(dinner_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df.iloc[k][0])):
        if dinner_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
            
    trigger = trigger / trigger.sum()
    result_dinner.append(trigger)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [52]:
np.isnan(result_dinner).sum()

1428

In [156]:
result_lunch_t = []

for k in range(len(lunch_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df_t.iloc[k][0])):
        if lunch_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
            
    trigger = trigger / trigger.sum()
    result_lunch_t.append(trigger)


In [157]:
result_dinner_t = []

for k in range(len(dinner_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df_t.iloc[k][0])):
        if dinner_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
            
    trigger = trigger / trigger.sum()
    result_dinner_t.append(trigger)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [158]:
result_dinner_t[0]

array([0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25,
       0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  ,
       0.  ])

# 메뉴 데이터 생성

In [73]:
menu.columns

Index(['메뉴', '곡류', '두류', '난류', '묵류', '어패류', '육류', '채소류', '해조류', '떡류',
       '양념.및.장류', '김치류', '만두류', '면류', '과일류', '밥류', '죽류', '덮밥국밥류', '비빔밥볶음밥류',
       '김.초.밥류', '국수류', '국탕류', '찌개류', '구이류', '무침류', '볶음류', '장아찌류', '전류', '조림류',
       '찜류', '튀김류', '샐러드류', '단품류', '유제품', '빵과자류', '음료.및.주류'],
      dtype='object')

In [159]:
column = list(menu.columns[1:-1])
lunch_sort = pd.DataFrame(result_lunch ,columns=column)
dinner_sort = pd.DataFrame(result_dinner, columns = column)
lunch_sort_t = pd.DataFrame(result_lunch_t, columns=column)
dinner_sort_t = pd.DataFrame(result_dinner_t, columns = column)

In [160]:
lunch_sort.head(2)

,곡류,두류,난류,묵류,어패류,육류,채소류,해조류,떡류,양념.및.장류,...,볶음류,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류
0,0.1,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.1,0.0
1,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0


In [161]:
dinner_sort.head(2)

,곡류,두류,난류,묵류,어패류,육류,채소류,해조류,떡류,양념.및.장류,...,볶음류,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류
0,0.1,0.1,0.0,0.0,0.000,0.1,0.000,0.1,0.0,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.10,0.0,0.0
1,0.0,0.0,0.0,0.0,0.125,0.0,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0


In [162]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '중식메뉴_split', '석식메뉴_split'],
      dtype='object')

# Train data set / Test data set

In [163]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '중식메뉴_split', '석식메뉴_split'],
      dtype='object')

In [164]:
train_lunch_columns = ['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율']

train_dinner_columns = [ '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율']

test_lunch_columns = ['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율']

test_dinner_columns = ['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '년', '월', '일', '주',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율']

In [165]:
train_lunch = train[train_lunch_columns]
test_lunch = test[test_lunch_columns]

train_dinner = train[train_lunch_columns]
test_dinner = test[test_dinner_columns]

In [166]:
#메뉴
train_lunch = pd.concat([train_lunch, lunch_sort], axis = 1)
train_dinner = pd.concat([train_dinner, dinner_sort], axis = 1)
test_lunch = pd.concat([test_lunch, lunch_sort_t], axis = 1)
test_dinner = pd.concat([test_dinner, dinner_sort_t], axis = 1)
print('done')

done


In [167]:
y_lunch = train['중식계']
y_dinner = train['석식계']

# 모델링

In [106]:
from sklearn.ensemble import RandomForestRegressor

# 대회 규칙
# 평가산식 : MAE(Mean Absolute Error)
lunch_model = RandomForestRegressor(criterion='mae')
dinner_model = RandomForestRegressor(criterion='mae')

## 중식

In [168]:
train_lunch[train_lunch['곡류'].isna()]

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,년,월,일,주,...,볶음류,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류
361,2,2839,79,259,0,0,2017,7,19,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1151,2,3021,112,283,1,123,2020,11,4,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
train_lunch[train_lunch['찜류'].isna()]

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,년,월,일,주,...,볶음류,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류
361,2,2839,79,259,0,0,2017,7,19,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1151,2,3021,112,283,1,123,2020,11,4,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
train_dinner[train_dinner['찜류'].isna()].shape

(42, 49)

In [128]:
train_dinner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 49 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   요일              1205 non-null   int64  
 1   본사정원수           1205 non-null   int64  
 2   본사휴가자수          1205 non-null   int64  
 3   본사출장자수          1205 non-null   int64  
 4   본사시간외근무명령서승인건수  1205 non-null   int64  
 5   현본사소속재택근무자수     1205 non-null   int32  
 6   년               1205 non-null   int64  
 7   월               1205 non-null   int64  
 8   일               1205 non-null   int64  
 9   주               1205 non-null   int64  
 10  출근              1205 non-null   int64  
 11  휴가비율            1205 non-null   float64
 12  출장비율            1205 non-null   float64
 13  야근비율            1205 non-null   float64
 14  재택비율            1205 non-null   float64
 15  곡류              1163 non-null   float64
 16  두류              1163 non-null   float64
 17  난류              1163 non-null   f